<a href="https://colab.research.google.com/github/lala991204/ML-self-study/blob/master/8_7_Introduction_Document_Clustering_%26_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Opinion Review 데이터 세트를 이용한 문서 군집화 수행

In [60]:
# 압축 파일을 google drive에 바로 풀어주는 코드(해당 압축 파일을 풀 저장공간 설정)
cd "/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/OpinosisDataset1.0"

/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/OpinosisDataset1.0


In [61]:
# 압축 파일 풀기
!unzip -qq "/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/OpinosisDataset1.0/topics.zip"

In [77]:
import pandas as pd
import glob ,os

path = '/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/OpinosisDataset1.0'   

# path로 지정한 디렉토리 밑에 있는 모든 (.txt).data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.txt.data"))    
filename_list = []
opinion_text = []

for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성 
    df = pd.read_table(file_,index_col=None, header=0,encoding='latin1')
    
    filename_ = file_.split('/')[-1]     # 앞서 설정한 경로들의 마지막 부분만 추출(파일명, 확장자명 포함된)
    filename = filename_.split('.')[0]   # 파일명만 추출

    filename_list.append(filename)
    opinion_text.append(df.to_string())    # 뒤에서 lower()를 적용하기 위해 문자열로 변환

# 파일명 리스트와 파일내용 리스트를  DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()       # '...' 표시되어 있어 보이지만, 내용 들어 있음.

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_netbook_1005ha,...


In [70]:
all_files

['/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/OpinosisDataset1.0/accuracy_garmin_nuvi_255W_gps.txt.data',
 '/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/OpinosisDataset1.0/bathroom_bestwestern_hotel_sfo.txt.data',
 '/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/OpinosisDataset1.0/battery-life_amazon_kindle.txt.data',
 '/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/OpinosisDataset1.0/battery-life_ipod_nano_8gb.txt.data',
 '/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/OpinosisDataset1.0/battery-life_netbook_1005ha.txt.data',
 '/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/OpinosisDataset1.0/buttons_amazon_kindle.txt.data',
 '/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/OpinosisDataset1.0/comfort_honda_accord_2008.txt.data',
 '/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/OpinosisDataset1.0/comfort_toyota_camry_2007.txt.data',
 '/content/drive/MyDrive/Colab Notebooks/github

In [83]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [78]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# 문자열에서 문장부호 제거하기
# ord() 함수: 특정 한 문자를 아스키 코드 값으로 변환.
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# 문자열 치환시, 한 두개라면 replace 사용, 그러나 많을 경우 translate 함수 활용하면 좋음.
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [100]:
 string.punctuation       # 문자열에서의 문장부호 모음

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [99]:
remove_punct_dict

{33: None,
 34: None,
 35: None,
 36: None,
 37: None,
 38: None,
 39: None,
 40: None,
 41: None,
 42: None,
 43: None,
 44: None,
 45: None,
 46: None,
 47: None,
 58: None,
 59: None,
 60: None,
 61: None,
 62: None,
 63: None,
 64: None,
 91: None,
 92: None,
 93: None,
 94: None,
 95: None,
 96: None,
 123: None,
 124: None,
 125: None,
 126: None}

In [107]:
document_df['opinion_text'][0]             # 문장부호 제거 전

"                                                                                                                                                                                                                                                               , and is very, very accurate .\n0                                                                                                                                                                           but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .\n1                                                                                                              This function is not accurate if you don't leave it in battery mode say, when you stop at the Cracker Barrell for lunch and to play one of those trangle games with the tees .\n2                                                                                                                                         

In [108]:
dd = document_df['opinion_text'][0].lower().translate(remove_punct_dict)      # 문장부호 제거 후(양 끝의 ''는 원래 없는 것임)
dd

'                                                                                                                                                                                                                                                                and is very very accurate \n0                                                                                                                                                                           but for the most part we find that the garmin software provides accurate directions whereever we intend to go \n1                                                                                                              this function is not accurate if you dont leave it in battery mode say when you stop at the cracker barrell for lunch and to play one of those trangle games with the tees \n2                                                                                                                                                  

In [110]:
dd1 = nltk.word_tokenize(dd)
dd1

['and',
 'is',
 'very',
 'very',
 'accurate',
 '0',
 'but',
 'for',
 'the',
 'most',
 'part',
 'we',
 'find',
 'that',
 'the',
 'garmin',
 'software',
 'provides',
 'accurate',
 'directions',
 'whereever',
 'we',
 'intend',
 'to',
 'go',
 '1',
 'this',
 'function',
 'is',
 'not',
 'accurate',
 'if',
 'you',
 'dont',
 'leave',
 'it',
 'in',
 'battery',
 'mode',
 'say',
 'when',
 'you',
 'stop',
 'at',
 'the',
 'cracker',
 'barrell',
 'for',
 'lunch',
 'and',
 'to',
 'play',
 'one',
 'of',
 'those',
 'trangle',
 'games',
 'with',
 'the',
 'tees',
 '2',
 'it',
 'provides',
 'immediate',
 'alternatives',
 'if',
 'the',
 'route',
 'from',
 'the',
 'online',
 'map',
 'program',
 'was',
 'inaccurate',
 'or',
 'blocked',
 'by',
 'an',
 'obstacle',
 '3',
 'ive',
 'used',
 'other',
 'gps',
 'units',
 'as',
 'well',
 'as',
 'gps',
 'built',
 'into',
 'cars',
 'and',
 'to',
 'this',
 'day',
 'nothing',
 'beats',
 'the',
 'accuracy',
 'of',
 'a',
 'garmin',
 'gps',
 '4',
 'it',
 'got',
 'me',
 'fro

In [111]:
LemTokens(dd1) # 위에서 이미 워드 토큰화된 것으로 표제어 추출을 하는 것임.

['and',
 'is',
 'very',
 'very',
 'accurate',
 '0',
 'but',
 'for',
 'the',
 'most',
 'part',
 'we',
 'find',
 'that',
 'the',
 'garmin',
 'software',
 'provides',
 'accurate',
 'direction',
 'whereever',
 'we',
 'intend',
 'to',
 'go',
 '1',
 'this',
 'function',
 'is',
 'not',
 'accurate',
 'if',
 'you',
 'dont',
 'leave',
 'it',
 'in',
 'battery',
 'mode',
 'say',
 'when',
 'you',
 'stop',
 'at',
 'the',
 'cracker',
 'barrell',
 'for',
 'lunch',
 'and',
 'to',
 'play',
 'one',
 'of',
 'those',
 'trangle',
 'game',
 'with',
 'the',
 'tee',
 '2',
 'it',
 'provides',
 'immediate',
 'alternative',
 'if',
 'the',
 'route',
 'from',
 'the',
 'online',
 'map',
 'program',
 'wa',
 'inaccurate',
 'or',
 'blocked',
 'by',
 'an',
 'obstacle',
 '3',
 'ive',
 'used',
 'other',
 'gps',
 'unit',
 'a',
 'well',
 'a',
 'gps',
 'built',
 'into',
 'car',
 'and',
 'to',
 'this',
 'day',
 'nothing',
 'beat',
 'the',
 'accuracy',
 'of',
 'a',
 'garmin',
 'gps',
 '4',
 'it',
 'got',
 'me',
 'from',
 'poin

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', ngram_range=(1,2), min_df=0.05,
                             max_df=0.85)

# opinion_text 컬럼 값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


In [85]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행하며, 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [86]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,2
1,bathroom_bestwestern_hotel_sfo,...,0
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1


In [87]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,...,0
30,room_holiday_inn_london,...,0
31,rooms_bestwestern_hotel_sfo,...,0
32,rooms_swissotel_chicago,...,0


In [89]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1
19,keyboard_netbook_1005ha,...,1
26,performance_netbook_1005ha,...,1
41,size_asus_netbook_1005ha,...,1
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,1
44,speed_windows7,...,1


In [90]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,2
5,buttons_amazon_kindle,...,2
8,directions_garmin_nuvi_255W_gps,...,2
9,display_garmin_nuvi_255W_gps,...,2
10,eyesight-issues_amazon_kindle,...,2
11,features_windows7,...,2
12,fonts_amazon_kindle,...,2
23,navigation_amazon_kindle,...,2
33,satellite_garmin_nuvi_255W_gps,...,2
34,screen_garmin_nuvi_255W_gps,...,2


In [91]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
13,food_holiday_inn_london,...,3
14,food_swissotel_chicago,...,3
15,free_bestwestern_hotel_sfo,...,3
20,location_bestwestern_hotel_sfo,...,3
21,location_holiday_inn_london,...,3
24,parking_bestwestern_hotel_sfo,...,3
27,price_amazon_kindle,...,3
28,price_holiday_inn_london,...,3
38,service_bestwestern_hotel_sfo,...,3
39,service_holiday_inn_london,...,3


In [92]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
6,comfort_honda_accord_2008,...,4
7,comfort_toyota_camry_2007,...,4
16,gas_mileage_toyota_camry_2007,...,4
17,interior_honda_accord_2008,...,4
18,interior_toyota_camry_2007,...,4
22,mileage_honda_accord_2008,...,4
25,performance_honda_accord_2008,...,4
29,quality_toyota_camry_2007,...,4
37,seats_honda_accord_2008,...,4
47,transmission_toyota_camry_2007,...,4


In [93]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

# 소속 클러스터를 cluster_label 컬럼 할당, cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
48,updates_garmin_nuvi_255W_gps,...,0
44,speed_windows7,...,0
43,speed_garmin_nuvi_255W_gps,...,0
42,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,0
41,size_asus_netbook_1005ha,...,0
36,screen_netbook_1005ha,...,0
35,screen_ipod_nano_8gb,...,0
34,screen_garmin_nuvi_255W_gps,...,0
33,satellite_garmin_nuvi_255W_gps,...,0


# 군집(Cluster)별 핵심 단어 추출하기

In [112]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)    # 군집 3개, word 피처가 4611개로 구성됨

# 각 행의 배열 값은 각 군집 내의 4611개의 피처의 위치가 개별 중심과 얼마나 가까운가를 상대 값으로 나타냄
# 0~1의 값, 1에 가까울수록 중심과 가까운 값임.
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


In [95]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명을 반환
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}

    # cluster_centers array의 값이 큰 순으로 정렬된 index 값 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리 값이 큰 순으로 값을 구하기 위함
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]

    # 개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num

        # cluster_centers_.argsort()[:,::-1]로 구한 index를 이용하여 top n 피처 단어를 구함
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes]

        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상대값 구함
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

        # clsuter_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_feature_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames

    return cluster_details

In [96]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items(): 
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features: ', cluster_detail['top_features'])
        print('Reviews 파일명: ', cluster_detail['filenames'][:7])
        print('===================================================')

In [98]:
feature_names = tfidf_vect.get_feature_names()

# 상위 10개
cluster_details = get_cluster_details(cluster_model = km_cluster, cluster_data = document_df,
                                      feature_names = feature_names, clusters_num = 3, top_n_features = 10)

print_cluster_details(cluster_details)

####### Cluster 0
Top features:  ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명:  ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps']
####### Cluster 1
Top features:  ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명:  ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007', 'mileage_honda_accord_2008', 'performance_honda_accord_2008']
####### Cluster 2
Top features:  ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명:  ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwester

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
